# Environmental expert (XGBoost): analysis & calibration

This notebook contains **diagnostics and post-hoc calibration** for the trained environmental (context) expert used in the thesis.

It is designed to be run **after** the training notebook (e.g., `01_env_xgboost_final_clean.ipynb`) has produced:

- `models/context/xgboost_hsi_model_FINAL_no_vespula.json`
- `models/context/feature_names_FINAL_no_vespula.csv`
- `models/context/species_mapping_FINAL_no_vespula.csv`
- *(optional but recommended)* `outputs/context_xgb/preds/test_predictions_2025_FINAL_no_vespula.parquet`

## What you will do here
1. Load the trained model + metadata  
2. Load saved probabilities (preferred) or re-compute probabilities from the 2025 parquet  
3. Run diagnostics (feature importance, confusion matrix, per-species metrics, calibration curves)  
4. Export calibrated probabilities for downstream fusion  

> Best practice: keep training and analysis separate so plots/calibration experiments do not accidentally change training artefacts.


## 1. Setup
Imports and basic utilities.


In [ ]:
from __future__ import annotations

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

from sklearn.metrics import (
    accuracy_score,
    log_loss,
    classification_report,
    confusion_matrix,
)

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.model_selection import GroupKFold
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression

sns.set_style("whitegrid")
plt.rcParams["figure.dpi"] = 120
np.random.seed(42)


## 2. Paths
Repo-relative paths (no absolute `/Users/...` paths). Adjust if your layout differs.


In [ ]:
from digital_naturalist.paths import load_paths

# Single source of truth for repo paths
P = load_paths("configs/paths.yaml")

REPO_ROOT = P["REPO_ROOT"]

MODEL_DIR = P["CONTEXT_MODEL_DIR"]
OUT_DIR   = P["OUT_CONTEXT_XGB"]

FIG_DIR     = OUT_DIR / "figures"
METRICS_DIR = OUT_DIR / "metrics"
PREDS_DIR   = OUT_DIR / "preds"

for d in (FIG_DIR, METRICS_DIR, PREDS_DIR):
    d.mkdir(parents=True, exist_ok=True)

MODEL_PATH       = MODEL_DIR / "xgboost_hsi_model_FINAL_no_vespula.json"
FEATURES_PATH    = MODEL_DIR / "feature_names_FINAL_no_vespula.csv"
SPECIES_MAP_PATH = MODEL_DIR / "species_mapping_FINAL_no_vespula.csv"

# Saved probabilities (preferred)
TEST_PREDS_PATH = PREDS_DIR / "test_predictions_2025_FINAL_no_vespula.parquet"

# Optional (only if you saved them in training notebook)
TRAIN_PREDS_PATH = PREDS_DIR / "train_predictions_with_hsi.parquet"
VAL_PREDS_PATH   = PREDS_DIR / "val_predictions_with_hsi.parquet"

# Raw parquets (only needed if you do feature-based or group-based recalibration)
HISTORICAL_PATH = P["GBIF_TRAIN_DIR"] / "observations_filtered_50m_accuracy.parquet"
TEST_2025_PATH  = P["GBIF_VAL_DIR"]   / "observations_filtered_50m_accuracy.parquet"

print("=== Context analysis (XGBoost) — resolved paths ===")
print("Repo root:       ", REPO_ROOT)
print("Model:           ", MODEL_PATH)
print("Feature names:   ", FEATURES_PATH)
print("Species mapping: ", SPECIES_MAP_PATH)
print("Test preds:      ", TEST_PREDS_PATH)
print("Train preds:     ", TRAIN_PREDS_PATH)
print("Val preds:       ", VAL_PREDS_PATH)
print("Historical raw:  ", HISTORICAL_PATH)
print("2025 raw:        ", TEST_2025_PATH)


## 3. Load model + metadata
Loads the trained XGBoost model, canonical feature order, and species index mapping.


In [ ]:
if not MODEL_PATH.exists():
    raise FileNotFoundError(f"Model not found: {MODEL_PATH}")
if not FEATURES_PATH.exists():
    raise FileNotFoundError(f"Feature list not found: {FEATURES_PATH}")
if not SPECIES_MAP_PATH.exists():
    raise FileNotFoundError(f"Species mapping not found: {SPECIES_MAP_PATH}")

# Load model
model = xgb.XGBClassifier()
model.load_model(MODEL_PATH)

# Feature list (order matters!)
feature_names = pd.read_csv(FEATURES_PATH)["feature"].tolist()

# Species mapping (order matters!)
species_map = pd.read_csv(SPECIES_MAP_PATH).sort_values("idx")
species_names = species_map["species"].tolist()
idx_to_species = dict(zip(species_map["idx"], species_map["species"]))
species_to_idx = dict(zip(species_map["species"], species_map["idx"]))
n_classes = len(species_names)

print("n_features:", len(feature_names))
print("n_classes:", n_classes)
print("species order:", species_names)


## 4. Load test labels and probabilities

**Preferred:** load saved test-set probabilities produced by the training notebook.

If those are not available, the fallback recomputes probabilities from the 2025 parquet using the saved feature list.


In [ ]:
def prob_col_for_species(species: str) -> str:
    return f"prob_{species.replace(' ', '_')}"

def get_prob_cols(df: pd.DataFrame) -> list[str]:
    cols = []
    for sp in species_names:
        c = prob_col_for_species(sp)
        if c in df.columns:
            cols.append(c)
    return cols

test_df = None  # will be set if available
test_preds = None

# ---- Preferred: load saved predictions parquet ----
if TEST_PREDS_PATH.exists():
    test_preds = pd.read_parquet(TEST_PREDS_PATH)
    prob_cols = get_prob_cols(test_preds)

    if len(prob_cols) != n_classes:
        raise ValueError(f"Expected {n_classes} prob columns in {TEST_PREDS_PATH.name}, found {len(prob_cols)}.")

    if "species" not in test_preds.columns:
        raise ValueError("Saved predictions parquet must contain a 'species' column.")

    y_test = test_preds["species"].map(species_to_idx).to_numpy()
    y_proba_test = test_preds[prob_cols].to_numpy()
    y_pred_test = y_proba_test.argmax(axis=1)

    print(f"Loaded saved test probabilities: {len(test_preds):,} rows")
else:
    print("Saved test predictions not found. Falling back to recomputing from 2025 parquet...")

    if not TEST_2025_PATH.exists():
        raise FileNotFoundError(
            f"2025 parquet not found: {TEST_2025_PATH}\n"
            "Either generate saved predictions from the training notebook, or make sure the 2025 parquet exists."
        )

    test_df = pd.read_parquet(TEST_2025_PATH).copy()
    test_df = test_df[test_df["species"].isin(species_names)].copy()

    missing = [f for f in feature_names if f not in test_df.columns]
    if missing:
        raise ValueError(
            "Cannot recompute probabilities because engineered features are missing in the parquet.\n"
            f"Missing {len(missing)} features (showing up to 20): {missing[:20]}\n"
            "Tip: prefer loading the saved predictions parquet produced by the training notebook."
        )

    X_test = test_df[feature_names].fillna(0).to_numpy()
    y_test = test_df["species"].map(species_to_idx).to_numpy()

    y_proba_test = model.predict_proba(X_test)
    y_pred_test = y_proba_test.argmax(axis=1)

    print(f"Recomputed test probabilities: {len(test_df):,} rows")

# Sanity check
print("y_proba_test shape:", y_proba_test.shape)
print("prob sum max abs error:", float(np.max(np.abs(y_proba_test.sum(axis=1) - 1.0))))


## 5. Metrics helpers
Top-k accuracy and **normalised** multiclass Brier score (so values are comparable to your thesis scale ~0.10).


In [ ]:
def top_k_accuracy(y_true: np.ndarray, y_proba: np.ndarray, k: int) -> float:
    topk = np.argsort(y_proba, axis=1)[:, -k:]
    return float(np.mean([y_true[i] in topk[i] for i in range(len(y_true))]))

def multiclass_brier_norm(y_true: np.ndarray, y_proba: np.ndarray, n_classes: int) -> float:
    # Normalised multiclass Brier: mean(sum_k (p_k - y_k)^2) / K
    Y = np.eye(n_classes)[y_true]
    return float(np.mean(np.sum((y_proba - Y) ** 2, axis=1)) / n_classes)

def per_species_brier(y_true: np.ndarray, y_proba: np.ndarray, species_names: list[str]) -> pd.DataFrame:
    # Binary Brier per class (OvR)
    rows = []
    for i, sp in enumerate(species_names):
        y_bin = (y_true == i).astype(int)
        p = y_proba[:, i]
        rows.append({"species": sp, "brier": float(np.mean((p - y_bin) ** 2))})
    return pd.DataFrame(rows).sort_values("brier")


## 6. Baseline performance (uncalibrated)
Classification report + log loss + Brier on the 2025 temporal holdout.


In [ ]:
print("Top-1:", accuracy_score(y_test, y_pred_test))
print("Top-3:", top_k_accuracy(y_test, y_proba_test, 3))
print("Top-5:", top_k_accuracy(y_test, y_proba_test, 5))
print("Log loss:", log_loss(y_test, y_proba_test, labels=list(range(n_classes))))
print("Brier (norm):", multiclass_brier_norm(y_test, y_proba_test, n_classes))

print("\nClassification report (test):")
print(classification_report(y_test, y_pred_test, target_names=species_names, zero_division=0))

brier_df = per_species_brier(y_test, y_proba_test, species_names)
brier_df.to_csv(METRICS_DIR / "brier_per_species_uncalibrated_test.csv", index=False)
print("\nSaved per-species Brier:", METRICS_DIR / "brier_per_species_uncalibrated_test.csv")


In [ ]:
# Computes per-class One-vs-Rest AUC and mean predicted probability of the true class
# Uses existing variables: y_test, y_proba_test, y_pred_test, species_names, n_classes, METRICS_DIR

from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from IPython.display import display


# Get precision/recall/F1/support as a dict
report_dict = classification_report(
    y_test, y_pred_test,
    target_names=species_names,
    zero_division=0,
    output_dict=True,
)

rows = []
for i, sp in enumerate(species_names):
    mask = (y_test == i)
    y_bin = mask.astype(int)  # 1 for class i, 0 otherwise
    p = y_proba_test[:, i]

    # OvR AUC (undefined if only one class present)
    if y_bin.sum() == 0 or y_bin.sum() == len(y_bin):
        auc = float("nan")
    else:
        auc = float(roc_auc_score(y_bin, p))

    # Mean p(true class | x) for samples whose true label is this class
    mean_p_true = float(np.mean(y_proba_test[mask, i])) if mask.any() else float("nan")

    # Binary Brier per class (OvR)
    brier = float(np.mean((p - y_bin) ** 2))

    rows.append({
        "species": sp,
        "N": int(report_dict[sp]["support"]),
        "precision": float(report_dict[sp]["precision"]),
        "recall": float(report_dict[sp]["recall"]),
        "f1": float(report_dict[sp]["f1-score"]),
        "auc": auc,
        "mean_p_true": mean_p_true,
        "brier": brier,
    })

per_species_metrics = pd.DataFrame(rows)
per_species_metrics.to_csv(METRICS_DIR / "per_species_metrics_uncalibrated_test.csv", index=False)
print("Saved:", METRICS_DIR / "per_species_metrics_uncalibrated_test.csv")

# Pretty view (rounded) for quick copying into LaTeX
display(
    per_species_metrics.assign(
        precision=lambda d: d["precision"].round(3),
        recall=lambda d: d["recall"].round(3),
        f1=lambda d: d["f1"].round(3),
        auc=lambda d: d["auc"].round(3),
        mean_p_true=lambda d: d["mean_p_true"].round(3),
        brier=lambda d: d["brier"].round(4),
    )
)

# LaTeX helper: print rows with the same rounding as your thesis table
print("\nLaTeX rows (copy into your tabular):\n")
for _, r in per_species_metrics.iterrows():
    sp_tex = f"\\textit{{{r['species']}}}"
    print(
        f"{sp_tex} & {int(r['N'])} "
        f"& {r['precision']:.3f} & {r['recall']:.3f} & {r['f1']:.3f} "
        f"& {r['auc']:.3f} & {r['mean_p_true']:.3f} & {r['brier']:.4f} \\\\"
    )


## 7. Feature importance
Uses the model’s built-in feature importance. This does not require any data.


In [ ]:
def analyze_feature_importance(model, feature_names, top_n=40, out_path=None) -> pd.DataFrame:
    importance = model.feature_importances_
    feat_imp_df = (
        pd.DataFrame({"feature": feature_names, "importance": importance})
        .sort_values("importance", ascending=False)
        .reset_index(drop=True)
    )

    print(f"Top {top_n} features:")
    for _, row in feat_imp_df.head(top_n).iterrows():
        print(f"{row['feature']:.<55} {row['importance']:>8.4f}")

    top = feat_imp_df.head(top_n).iloc[::-1]
    fig, ax = plt.subplots(figsize=(10, max(6, 0.25 * top_n)))
    ax.barh(top["feature"], top["importance"])
    ax.set_xlabel("Importance")
    ax.set_title(f"Top {top_n} feature importances")
    plt.tight_layout()

    if out_path is None:
        out_path = FIG_DIR / "feature_importance_top.png"
    fig.savefig(out_path, dpi=300, bbox_inches="tight")
    print("Saved:", out_path)

    return feat_imp_df

feat_imp_df = analyze_feature_importance(model, feature_names, top_n=40)
feat_imp_df.to_csv(METRICS_DIR / "feature_importance.csv", index=False)
print("Saved:", METRICS_DIR / "feature_importance.csv")


## 8. Confusion matrix
Counts + row-normalised heatmap.


In [ ]:
cm = confusion_matrix(y_test, y_pred_test, labels=list(range(n_classes)))
cm_norm = cm.astype(float) / np.maximum(cm.sum(axis=1, keepdims=True), 1)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    cm_norm,
    annot=cm,
    fmt="d",
    cmap="Blues",
    xticklabels=[s.replace(" ", "\n") for s in species_names],
    yticklabels=[s.replace(" ", "\n") for s in species_names],
    cbar_kws={"label": "Row-normalized proportion"},
    ax=ax,
    square=True
)

ax.set_xlabel("Predicted")
ax.set_ylabel("True")
ax.set_title("Confusion matrix (counts annotated; colours row-normalised)")
plt.tight_layout()

out_path = FIG_DIR / "confusion_matrix_test.png"
fig.savefig(out_path, dpi=300, bbox_inches="tight")
print("Saved:", out_path)


## 9. Calibration curves (reliability diagrams)
Per-species curves using quantile binning.


In [ ]:
def plot_calibration_curves(y_true, y_proba, species_names, n_bins=10, out_path=None, title=None):
    n_species = len(species_names)
    n_cols = 3
    n_rows = (n_species + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = np.array(axes).reshape(-1)

    for i, sp in enumerate(species_names):
        ax = axes[i]
        y_bin = (y_true == i).astype(int)
        p = y_proba[:, i]

        prob_true, prob_pred = calibration_curve(y_bin, p, n_bins=n_bins, strategy="quantile")
        ax.plot(prob_pred, prob_true, marker="o", linewidth=2)
        ax.plot([0, 1], [0, 1], "k--", alpha=0.5)

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.grid(alpha=0.3)
        ax.set_title(sp.replace(" ", "\n"), fontsize=9, fontweight="bold")
        ax.set_xlabel("Mean predicted probability")
        ax.set_ylabel("Fraction of positives")

    for j in range(n_species, len(axes)):
        axes[j].axis("off")

    if title:
        fig.suptitle(title, y=1.02)

    plt.tight_layout()

    if out_path is not None:
        fig.savefig(out_path, dpi=300, bbox_inches="tight")
        print("Saved:", out_path)

    return fig

_ = plot_calibration_curves(
    y_test, y_proba_test, species_names, n_bins=10,
    out_path=FIG_DIR / "calibration_curves_uncalibrated_test.png",
    title="Calibration curves (uncalibrated) — test"
)


## 10. Notes for thesis reporting

- Report calibration metrics on the **2025 temporal holdout** (test set).  
- Use **normalised** multiclass Brier score for comparability across number of classes.  
- If isotonic improves both log loss and Brier, prefer isotonic over sigmoid.  
- Keep large artefacts (`data/`, `models/`, `outputs/`) gitignored in the public repo.
